# Local-Llama-Inference - Streaming Responses

Demonstrates token-by-token streaming for real-time text generation.

## Features
- **Streaming Chat**: See tokens as they're generated
- **Real-time Output**: Perfect for interactive applications
- **Python Generators**: Use Python's async/await patterns
- **Low Latency**: Immediate feedback

In [ ]:
from local_llama_inference import LlamaServer, LlamaClient, detect_gpus
from pathlib import Path
from huggingface_hub import hf_hub_download

print("✅ Package imported")

## Download Model

In [ ]:
models_dir = Path.home() / "models"
models_dir.mkdir(exist_ok=True)

# Download model if not already present
model_path = hf_hub_download(
    repo_id="TheBloke/phi-2-GGUF",
    filename="phi-2.Q4_K_M.gguf",
    local_dir=str(models_dir),
)

print(f"✅ Model ready: {model_path}")

## Start Server

In [ ]:
print("🚀 Starting server...")
server = LlamaServer(
    model_path=model_path,
    n_gpu_layers=33,
    n_threads=4,
)
server.start()
server.wait_ready(timeout=60)
print(f"✅ Server ready")

client = LlamaClient()

## Example 1: Basic Streaming Chat

In [ ]:
print("🤖 Streaming response (watch tokens appear in real-time):\n")
print("Q: Write a haiku about machine learning\nA: ", end="", flush=True)

# Stream responses token by token
for chunk in client.stream_chat(
    messages=[
        {"role": "user", "content": "Write a haiku about machine learning"}
    ],
    max_tokens=100,
):
    # Extract token from response
    token = chunk.choices[0].delta.content
    if token:
        print(token, end="", flush=True)

print("\n")

## Example 2: Streaming Text Completion

In [ ]:
print("🤖 Streaming completion:\n")
print("Prompt: The future of AI is...\nResponse: ", end="", flush=True)

prompt = "The future of AI is"
for chunk in client.stream_complete(
    prompt=prompt,
    max_tokens=150,
    temperature=0.8,
):
    token = chunk.choices[0].text
    if token:
        print(token, end="", flush=True)

print("\n")

## Example 3: Measuring Latency

In [ ]:
import time

print("⏱️  Measuring token generation latency:\n")

start_time = time.time()
token_count = 0
first_token_time = None

print("Prompt: Explain quantum computing\nResponse: ", end="", flush=True)

for chunk in client.stream_chat(
    messages=[
        {"role": "user", "content": "Explain quantum computing in one sentence"}
    ],
    max_tokens=100,
):
    token = chunk.choices[0].delta.content
    if token:
        token_count += 1
        if first_token_time is None:
            first_token_time = time.time() - start_time
        print(token, end="", flush=True)

total_time = time.time() - start_time

print("\n")
print(f"\n📊 Latency Metrics:")
print(f"  First token latency: {first_token_time*1000:.1f} ms")
print(f"  Tokens generated: {token_count}")
print(f"  Total time: {total_time:.2f} seconds")
if total_time > 0:
    print(f"  Tokens/second: {token_count/total_time:.2f}")

## Example 4: Batch Multiple Streaming Requests

In [ ]:
print("🔄 Streaming multiple requests sequentially:\n")

prompts = [
    "What is Python?",
    "What is CUDA?",
    "What is GPU computing?",
]

for i, prompt in enumerate(prompts, 1):
    print(f"\n[{i}] Q: {prompt}")
    print("    A: ", end="", flush=True)
    
    for chunk in client.stream_chat(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=80,
    ):
        token = chunk.choices[0].delta.content
        if token:
            print(token, end="", flush=True)
    
    print()  # Newline

## Example 5: Interactive Streaming (Multi-turn)

In [ ]:
print("💬 Interactive streaming conversation:\n")

messages = [
    {"role": "system", "content": "You are a helpful AI assistant who speaks concisely."}
]

# Turn 1
user_input = "What is recursion in programming?"
print(f"👤 User: {user_input}")
print("🤖 Assistant: ", end="", flush=True)

messages.append({"role": "user", "content": user_input})
assistant_response = ""

for chunk in client.stream_chat(
    messages=messages,
    max_tokens=100,
):
    token = chunk.choices[0].delta.content
    if token:
        assistant_response += token
        print(token, end="", flush=True)

print("\n")
messages.append({"role": "assistant", "content": assistant_response})

# Turn 2
user_input = "Can you give an example?"
print(f"👤 User: {user_input}")
print("🤖 Assistant: ", end="", flush=True)

messages.append({"role": "user", "content": user_input})
assistant_response = ""

for chunk in client.stream_chat(
    messages=messages,
    max_tokens=100,
):
    token = chunk.choices[0].delta.content
    if token:
        assistant_response += token
        print(token, end="", flush=True)

print()

## Stop Server

In [ ]:
print("\n🛑 Stopping server...")
server.stop()
print("✅ Done")

## Key Points

- **Streaming**: Use `stream_chat()` or `stream_complete()` for token-by-token generation
- **Python Generators**: Iterate with `for` loop over chunks
- **Low Latency**: First token appears quickly, visible feedback
- **Multi-turn**: Build conversation history as you stream
- **Parameters**: All standard parameters work with streaming (temperature, max_tokens, etc.)

## Next Notebooks

- `03_embeddings.ipynb` - Generate text embeddings
- `04_multi_gpu.ipynb` - Multi-GPU tensor parallelism
- `05_advanced_api.ipynb` - All 30+ API endpoints